<a href="https://colab.research.google.com/github/ai-fast-track/icevision-gradio/blob/master/icevision_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install git+git://github.com/airctic/mantisshrimp.git#egg=mantisshrimp[inference] --upgrade

  Cloning git://github.com/airctic/mantisshrimp.git to /tmp/pip-install-b7hxnuiw/mantisshrimp
  Running command git clone -q git://github.com/airctic/mantisshrimp.git /tmp/pip-install-b7hxnuiw/mantisshrimp
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 122kB 10.3MB/s 
     - 337kB 10.7MB/s
     |████████████████████████████████| 952kB 28.5MB/s 
     |████████████████████████████████| 225kB 33.3MB/s 
  Created wheel for mantisshrimp: filename=mantisshrimp-0.0.9rc1-cp36-none-any.whl size=116192 sha256=16b4c0612f237869d58bef8671f71a2edfd6cd8f54959bc91cb5b5acca415e66
  Stored in directory: /tmp/pip-ephem-wheel-cache-c8a14tqi/wheels/f2/de/15/b479b63a90d19b1b6b5f6011eac5da5aa8fd30858d708899a0
  Created wheel for albumentations: filename=albumentations-0.4.6-cp36-none-any.whl size=65165 sha256=f63c0e88f4c5bf18349cf198d723b9de434165d0797aa2fee71fcb24a234c890
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4ad

In [2]:
pip install gradio

     |████████████████████████████████| 1.0MB 4.8MB/s 
     |████████████████████████████████| 215kB 26.7MB/s 
     |████████████████████████████████| 2.6MB 18.8MB/s 
     |████████████████████████████████| 71kB 7.5MB/s 
     |████████████████████████████████| 962kB 45.5MB/s 


In [3]:
from mantisshrimp.all import *
import PIL, requests
import torch
from torchvision import transforms
import gradio as gr

In [4]:
MASK_PENNFUNDAN_WEIGHTS_URL = "https://mantisshrimp-models.s3.us-east-2.amazonaws.com/pennfundan_maskrcnn_resnet50fpn.zip"
# img_url = "https://raw.githubusercontent.com/ai-fast-track/ice-streamlit/master/images/image2.png"

In [5]:
def load_model(class_map=class_map, url=None):
    if url is None:
        # print("Please provide a valid URL")
        return None
    else:
        model = mask_rcnn.model(num_classes=len(class_map))
        state_dict = torch.hub.load_state_dict_from_url(
            url, map_location=torch.device("cpu")
        )
        model.load_state_dict(state_dict)
        return model

In [7]:
class_map = datasets.pennfundan.class_map()
model = load_model(class_map=class_map, url=MASK_PENNFUNDAN_WEIGHTS_URL)
print("class_map: ", class_map)

class_map:  <ClassMap: {'background': 0, 'person': 1}>


In [18]:
def predict(
    model, image, detection_threshold: float = 0.5, mask_threshold: float = 0.5
):
    # img = np.array(image)
    tfms_ = tfms.A.Adapter([tfms.A.Normalize()])
    # Whenever you have images in memory (numpy arrays) you can use `Dataset.from_images`
    infer_ds = Dataset.from_images([image], tfms_)

    batch, samples = mask_rcnn.build_infer_batch(infer_ds)
    preds = mask_rcnn.predict(
        model=model,
        batch=batch,
        detection_threshold=detection_threshold,
        mask_threshold=mask_threshold,
    )
    return samples[0]["img"], preds[0]

def get_masks(input_image):
    img, pred = predict(model=model, image=input_image)
    # print(pred)
    img = draw_pred(img=img, pred=pred, class_map=class_map, denormalize_fn=denormalize_imagenet)
    img = PIL.Image.fromarray(img)
    print("Output Image: ", img.size, type(img))
    return img

In [ ]:
outputs = gr.outputs.Image(type="pil")
gr_interface = gr.Interface(fn=get_masks, inputs="image", outputs=outputs, title='IceVision - Ice Simple')
gr_interface.launch(inline=False, share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 6 hours. If you need a permanent link, email support@gradio.app
Running on External URL: https://31671.gradio.app
Output Image:  (800, 535) <class 'PIL.Image.Image'>
Output Image:  (600, 400) <class 'PIL.Image.Image'>
